In [5]:
# Importing neccessary packages
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import json
#from tensorflow.keras.utils import  get_data_generators, plot_model_results
#from Models import TrainingCheckpoint, ModelTrain

**Unzip the file**

In [7]:
from zipfile import ZipFile
file_name = "/content/drive/MyDrive/fer2013.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print("Done")

Done


**Import the Libraries**

In [8]:
import numpy as np
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:
NUM_CLASSES = 7
classes_list = ["angry","disgust","fear","happy","neutral","sad","surprise"]

**Initializing Training And Test Generators:**

In [9]:
# Data augmentation for regularization and adding some extra training samples

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg19.preprocess_input,
    rescale=1.0/255.0,
    horizontal_flip=True, 
    zoom_range=0.2,
    shear_range=0.2
)

In [11]:
TARGET_DIM = 300
BATCH_SIZE = 32

In [52]:
TRAIN_DIR = 'train'
datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255)
train_generator = datagen.flow_from_directory(
    TRAIN_DIR, 
    target_size=(224,224),
    subset='training',
    batch_size=32,
    class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(224,224),
    subset='validation',
    batch_size=32
)

Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.


In [53]:
train_generator.class_indices

{'angry': 0,
 'disgust': 1,
 'fear': 2,
 'happy': 3,
 'neutral': 4,
 'sad': 5,
 'surprise': 6}

In [54]:
validation_generator.class_indices

{'angry': 0,
 'disgust': 1,
 'fear': 2,
 'happy': 3,
 'neutral': 4,
 'sad': 5,
 'surprise': 6}

In [55]:
base_model = tf.keras.applications.vgg19.VGG19(
    include_top=False, 
    weights='imagenet', 
    input_shape=(TARGET_DIM, TARGET_DIM, 3)
)

In [56]:
print('Layers in Vgg19: ' + str(len(base_model.layers)))

Layers in Vgg19: 22


In [57]:
preds = base_model.output
preds = tf.keras.layers.GlobalAveragePooling2D()(preds)
preds = tf.keras.layers.Dense(1024, activation=tf.nn.relu)(preds)
preds = tf.keras.layers.BatchNormalization()(preds)
preds = tf.keras.layers.Dense(512, activation=tf.nn.relu)(preds)
preds = tf.keras.layers.BatchNormalization()(preds)
preds = tf.keras.layers.Dense(256, activation=tf.nn.relu)(preds)
preds = tf.keras.layers.BatchNormalization()(preds)
preds = tf.keras.layers.Dense(128, activation=tf.nn.relu)(preds)
preds = tf.keras.layers.Dense(7, activation=tf.nn.softmax)(preds)

In [58]:
model = tf.keras.models.Model(base_model.input, preds)

In [59]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 300, 300, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 300, 300, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 300, 300, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 150, 150, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 150, 150, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 150, 150, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 75, 75, 128)       0   

In [43]:
class ModelTrain:

    def __init__(self, model, target_dim, batch_size):
        self.model = model
        self.target_dim = target_dim
        self.batch_size = batch_size
     
    def freeze_layers(self, no_layers):
        
        for layers in self.model.layers[:-no_layers]:
            layers.trainable = False

        for layers in self.model.layers[-no_layers:]:
            layers.trainable = True 
        
    def train(self, train_generator, validation_generator, callbacks = [], epochs = 50):
        
        self.model.fit_generator(
            train_generator,
            steps_per_epoch=train_generator.samples // self.batch_size,
            validation_data=validation_generator,
            validation_steps=validation_generator.samples // self.batch_size,
            callbacks=callbacks,
            epochs=epochs
        )
 

# In[3]:


class TrainingCheckpoint(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if logs['acc'] > 0.90:
            print("Reached target training accuracy")
            self.model.stop_training = True

In [60]:
vgg19 = ModelTrain(model, TARGET_DIM, BATCH_SIZE)
vgg19.freeze_layers(9)

In [61]:
vgg19.model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 300, 300, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 300, 300, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 300, 300, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 150, 150, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 150, 150, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 150, 150, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 75, 75, 128)       0   

In [62]:
vgg19.model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['acc']
)

In [63]:
# Lets define checkpoint for model saving
filepath="./models/vgg19-{epoch:02d}-{acc:.2f}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')

training_checkpoint = TrainingCheckpoint()

csv_logger = tf.keras.callbacks.CSVLogger(filename='./logs/vgg19_training.csv', append=True)

In [64]:
vgg19.model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    callbacks=[checkpoint, csv_logger, training_checkpoint],
    epochs=100
)

Epoch 1/100


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


  4/717 [..............................] - ETA: 4:00:17 - loss: 2.3224 - acc: 0.2109

KeyboardInterrupt: ignored